In [42]:
!pip install -q gradio pydub sarvamai

import gradio as gr
from pydub import AudioSegment
import tempfile, base64, traceback
from sarvamai import SarvamAI


In [43]:
import getpass
from sarvamai import SarvamAI
from sarvamai.play import save
SARVAM_API_KEY = getpass.getpass("🔑 Enter your Sarvam API key: ")
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)


🔑 Enter your Sarvam API key: ··········


In [44]:
conversation_history = [
    {"role": "system", "content": "You are a friendly, bilingual voice assistant that replies naturally and politely."}
]

def voice_chat(audio_file, chat_history):
    global conversation_history
    try:
        audio = AudioSegment.from_file(audio_file)
        audio = audio.set_frame_rate(16000).set_channels(1)
        audio.export("input.wav", format="wav")
        print("🎧 Converted to input.wav")

        stt = client.speech_to_text.transcribe(
            file=open("input.wav", "rb"),
            model="saarika:v2.5",
            language_code="unknown"
        )
        detected_lang = getattr(stt, "language_code", None) or "en-IN"
        if detected_lang == "unknown":
            detected_lang = "en-IN"

        print("🧾 STT response:", stt)

        transcript = ""
        if hasattr(stt, "text"):
            transcript = stt.text
        elif hasattr(stt, "transcript"):
            transcript = stt.transcript
        elif isinstance(stt, dict):
            transcript = stt.get("text") or stt.get("transcript", "")
        transcript = (transcript or "").strip()

        if not transcript:
            chat_history += [["❌ Couldn't understand audio.", None]]
            return chat_history, None

        print("🗣️ User said:", transcript)

        conversation_history.append({"role": "user", "content": transcript})
        chat_response = client.chat.completions(
            messages=conversation_history,
            temperature=0.5,
            top_p=1,
            max_tokens=1000,
        )
        reply = chat_response.choices[0].message.content
        conversation_history.append({"role": "assistant", "content": reply})

        if len(conversation_history) > 7:
            conversation_history = [conversation_history[0]] + conversation_history[-6:]

        tts_response = client.text_to_speech.convert(
            target_language_code= detected_lang,
            text=reply,
            model="bulbul:v2",
            speaker="anushka"
        )

        if hasattr(tts_response, "audios") and len(tts_response.audios) > 0:
            audio_base64 = tts_response.audios[0]
            tts_audio = base64.b64decode(audio_base64)
        else:
            chat_history += [[f"❌ TTS failed: {tts_response}", None]]
            return chat_history, None

        tts_path = tempfile.mktemp(suffix=".wav")
        with open(tts_path, "wb") as f:
            f.write(tts_audio)

        chat_history.append((
        gr.Audio(audio_file, label="🎤 You"),
        gr.Audio(tts_path, label="🤖 AI")
))

        return chat_history, tts_path

    except Exception as e:
        traceback.print_exc()
        chat_history += [[f"❌ Error: {str(e)}", None]]
        return chat_history, None

def end_conversation():
    global conversation_history
    conversation_history = [
        {"role": "system", "content": "You are a friendly, bilingual voice assistant that replies naturally and politely."}
    ]
    return [], None


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        <div style='text-align:center;'>
        <h2>💬 Indic Voice Conversational Agent</h2>
        <p>🎙️ Send a voice note — get a voice reply back!</p>
        </div>
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=500, label="🎙️ Voice Conversation", show_label=True)
        with gr.Column(scale=1):
            mic = gr.Audio(type="filepath", label="🎤 Record your voice note")
            audio_out = gr.Audio(label="🔊 Model reply playback")

    with gr.Row():
        send_btn = gr.Button("Send Voice Note")
        end_btn = gr.Button("End Conversation")

    send_btn.click(fn=voice_chat, inputs=[mic, chatbot], outputs=[chatbot, audio_out])
    end_btn.click(fn=end_conversation, outputs=[chatbot, audio_out])

demo.launch(share=True)


/tmp/ipython-input-3367980720.py:102: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, label="🎙️ Voice Conversation", show_label=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7964bda788c88bc78e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
